In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

In [ ]:
startpath = '/content/drive/MyDrive/대출 데이터 모음/작업 폴더'
list_files(startpath)

작업 폴더/
    2018 데이터 가공/
        2018_1월.xlsx
        2018_3월.xlsx
        2018_2월.xlsx
        2018_5월.xlsx
        2018_6월.xlsx
        2018_4월.xlsx
        2018_9월.xlsx
        2018_7월.xlsx
        2018_8월.xlsx
        2018_10월.xlsx
        2018_11월.xlsx
        2018_12월.xlsx
    2019 데이터 가공/
        2019_1월.xlsx
        2019_2월.xlsx
        2019_3월.xlsx
        2019_4월.xlsx
        2019_5월.xlsx
        2019_6월.xlsx
        2019_7월.xlsx
        2019_8월.xlsx
        2019_9월.xlsx
        2019_10월.xlsx
        2019_11월.xlsx
        2019_12월.xlsx
    2020 데이터 가공/
    2021 데이터 가공/
        2021_1월.xlsx
        2021_2월.xlsx
        2021_3월.xlsx
        2021_5월.xlsx
        2021_6월.xlsx
        2021_7월.xlsx
        2021_8월.xlsx
        2021_9월.xlsx
        2021_10월.xlsx
        2021_11월.xlsx
        2021_12월.xlsx
        2021_4월.xlsx
    2022 데이터 가공/
        2022_2월.xlsx
        2022_4월.

In [ ]:
import pandas as pd
def filtering_datarow(file_path, year, month):
  # 엑셀 파일 불러오기
  df = pd.read_excel(file_path)

  # 조건에 따라 행 제거
  # 1. "Why?" 시리즈 제거
  df = df[~df['서명'].str.contains("Why?")]

  # 2. 3D 프린팅 및 시각 연출 관련 서적 제거 (KDC 004.76)
  df = df[~df['KDC'].astype(str).str.startswith('4.76')]

  # 3. 유튜브, SNS 등과 같은 서적 제거 (004.77)
  df = df[~df['KDC'].astype(str).str.startswith('4.77')]

  # 4. 컴퓨터 활용 능력 관련 고시 서적 제거 (004.077)
  df = df[~df['KDC'].astype(str).str.startswith('4.077')]

  df = df[~df['KDC'].astype(str).str.startswith('1')]

  # 5. 만화, 퀴즈와 같은 키워드 제거 (제목 탐색)
  df = df[~df['서명'].str.contains("만화")]

  # 6. 005.5에 해당되는 데이터 예외 처리
  # 005.54 (통계 프로그램) 제외, 005.58 (모바일 앱) 제외, "노션" 포함 제거
  df = df[~(df['KDC'].astype(str).str.startswith('5.5') & ~df['KDC'].astype(str).str.startswith(('5.54', '5.58')))]
  df = df[~(df['서명'].str.contains('노션') & df['KDC'].astype(str).str.startswith('5.58'))]

  # 이미 수행한 결과이므로 처리하지 않음

  df = df[~df['서명'].str.contains("블로그")]
  df = df[~df['서명'].str.contains("유튜브")]
  df = df[~df['서명'].str.contains("어린이")]
  df = df[~df['서명'].str.contains("청소년")]

  # 결과 저장
  df.to_excel(f'/content/drive/MyDrive/대출 데이터 모음/결과 폴더/{year}/{year}_{month}.xlsx', index=False)


In [ ]:
years = range(2018, 2024)  # 연도 범위
months = range(1, 13)  # 월 범위


# 각 연도별 데이터 폴더를 순회
base_path = '/content/drive/MyDrive/대출 데이터 모음/데이터 소스 폴더/'
for year in years:
    year_folder_path = os.path.join(base_path, str(year))
    if os.path.exists(year_folder_path):
        for month in months:
            file_name = f"{year}_{month}.xlsx"
            file_path = os.path.join(year_folder_path, file_name)
            if os.path.exists(file_path):
                filtering_datarow(file_path, year, month)
            else:
                print(f"File not found: {file_path}")
    else:
        print(f"Year folder not found: {year_folder_path}")

KeyboardInterrupt: ignored

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.3 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
import re

def extract_keywords(text):
    # KoNLPy 형태소 분석기
    okt = Okt()

    # 한국어 명사 추출
    nouns = okt.nouns(text)

    # 영어 단어 추출 (정규 표현식 사용)
    english_words = re.findall(r'\b[A-Za-z]+(?:\s+[A-Za-z]+)*\b', text)

    # 중복 제거 및 결과 병합
    keywords = set(nouns) | set(english_words)

    return keywords

# 제목 분석
title = "(자바 ORM 표준) JPA 프로그래밍 :스프링 데이터 예제 프로젝트로 배우는 전자정부 표준 데이터베이스 프레임워크"
keywords = extract_keywords(title)
print(keywords)


{'프레임워크', '데이터', '프로젝트', '데이터베이스', '스프링', '자바', '프로그래밍', 'ORM', 'JPA', '전자정부', '예제', '표준'}


In [ ]:
from konlpy.tag import Okt
import re

def extract_keywords(text, stop_words):
    # KoNLPy 형태소 분석기
    okt = Okt()

    # 한국어 명사 추출
    nouns = okt.nouns(text)

    # 영어 단어 추출 (정규 표현식 사용)
    english_words = re.findall(r'\b[A-Za-z]+(?:\s+[A-Za-z]+)*\b', text)

    # 중복 제거 및 결과 병합
    keywords = set(nouns) | set(english_words)

    # 불용어 제거
    filtered_keywords = [word for word in keywords if word.lower() not in stop_words]

    return filtered_keywords

# 불용어 사전 정의 (예시)
stop_words = set(['배우', '표준', '사용'])

# 제목 분석
title = "(자바 ORM 표준) JPA 프로그래밍 :스프링 데이터 예제 프로젝트로 배우는 전자정부 표준 데이터베이스 프레임워크"
keywords = extract_keywords(title, stop_words)
print(keywords)


['JPA', '전자정부', '데이터', '프레임워크', '스프링', '자바', 'ORM', '데이터베이스', '프로그래밍']


In [ ]:
from konlpy.tag import Okt
import re

def extract_keywords(text):
    # KoNLPy 형태소 분석기
    okt = Okt()

    # 한국어 명사 추출
    nouns = okt.nouns(text)

    # 영어 단어 추출 (정규 표현식 사용)
    english_words = re.findall(r'\b[A-Za-z]+(?:\s+[A-Za-z]+)*\b', text)

    # 중복 제거 및 결과 병합
    keywords = set(nouns) | set(english_words)

    # 불용어 제거
    filtered_keywords = [word for word in keywords if word.lower() not in stop_words]

    return filtered_keywords










['장도', '노하우', '기획', '웹', '팀', '처음', '다시']


In [ ]:
def keyword_processing(file_path, year, month):
  temp = pd.read_excel(file_path)
  temp['제목-키워드'] = temp['서명'].apply(extract_keywords)
  temp.to_excel(f'/content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/{year}/{year}_{month}_keyword_processed.xlsx')

# 제목에서 키워드 추출

In [ ]:
years = range(2018, 2024)  # 연도 범위
months = range(1, 13)  # 월 범위


# 각 연도별 데이터 폴더를 순회
base_path = '/content/drive/MyDrive/대출 데이터 모음/결과 폴더/'
for year in years:
    year_folder_path = os.path.join(base_path, str(year))
    if os.path.exists(year_folder_path):
        for month in months:
            file_name = f"{year}_{month}.xlsx"
            file_path = os.path.join(year_folder_path, file_name)
            if os.path.exists(file_path):
                # 파일을 가지고 수정만 하면 됨
                keyword_processing(file_path, year, month)
            else:
                print(f"File not found: {file_path}")
    else:
        print(f"Year folder not found: {year_folder_path}")

File not found: /content/drive/MyDrive/대출 데이터 모음/결과 폴더/2023/2023_11.xlsx
File not found: /content/drive/MyDrive/대출 데이터 모음/결과 폴더/2023/2023_12.xlsx


# 제목-키워드에서 빈도가 많은 키워드 추출

In [ ]:
def update_keyword_counts(keyword_dict, file_path):
    import pandas as pd

    # 파일에서 데이터 프레임을 읽어옴
    df = pd.read_excel(file_path)

    # '제목-키워드' 열의 각 행을 순회하면서 키워드 존재 여부 확인 및 카운트 업데이트
    for keywords in df['제목-키워드']:
        for keyword in keywords:
            if keyword in keyword_dict:
                keyword_dict[keyword] += 1
            else:
                # 키워드가 딕셔너리에 없으면 새로 추가하고 카운트를 1로 설정
                keyword_dict[keyword] = 1

    return keyword_dict

In [ ]:
years = range(2018, 2024)  # 연도 범위
months = range(1, 13)  # 월 범위


# 각 연도별 데이터 폴더를 순회
base_path = '/content/drive/MyDrive/대출 데이터 모음/결과 폴더/'
for year in years:
    year_folder_path = os.path.join(base_path, str(year))
    if os.path.exists(year_folder_path):
        for month in months:
            file_name = f"{year}_{month}.xlsx"
            file_path = os.path.join(year_folder_path, file_name)
            if os.path.exists(file_path):
                # 파일을 가지고 수정만 하면 됨
                keyword_processing(file_path, year, month)
            else:
                print(f"File not found: {file_path}")
    else:
        print(f"Year folder not found: {year_folder_path}")

,제목,제목-키워드
0,(자바 ORM 표준) JPA 프로그래밍,"[표준, JPA, 자바, ORM, 프로그래밍]"
1,파이썬 데이터 분석,"[데이터, 파이썬, 분석]"
2,인공지능과 머신러닝 입문,"[러닝, 입문, 지능, 머신, 인공]"


# 키워드 추출

In [ ]:
import ast

def update_keyword_counts(keyword_dict, file_path):
    # 파일에서 데이터 프레임을 읽어옴
    df = pd.read_excel(file_path)

    # '제목-키워드' 열의 각 행을 순회하면서 키워드 존재 여부 확인 및 카운트 업데이트
    for keywords_str in df['제목-키워드']:
        # 문자열을 리스트로 변환
        try:
            keywords = ast.literal_eval(keywords_str)
            if isinstance(keywords, list):
                for keyword in keywords:
                    if keyword in keyword_dict:
                        keyword_dict[keyword] += 1
                    else:
                        keyword_dict[keyword] = 1
        except:
            continue  # 변환 실패 시 다음 행으로 이동

    return keyword_dict

In [ ]:
years = range(2018, 2024)  # 연도 범위
months = range(1, 13)  # 월 범위

keyword_dict = dict()

# 각 연도별 데이터 폴더를 순회
base_path = '/content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/'
for year in years:
    year_folder_path = os.path.join(base_path, str(year))
    if os.path.exists(year_folder_path):
        for month in months:
            file_name = f"{year}_{month}_keyword_processed.xlsx"
            file_path = os.path.join(year_folder_path, file_name)
            if os.path.exists(file_path):
                # 파일을 가지고 수정만 하면 됨
                update_keyword_counts(keyword_dict, file_path)
            else:
                print(f"File not found: {file_path}")
    else:
        print(f"Year folder not found: {year_folder_path}")

File not found: /content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/2023/2023_11_keyword_processed.xlsx
File not found: /content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/2023/2023_12_keyword_processed.xlsx


In [ ]:
keyword_dict

{'관': 125,
 '알고리즘': 3035,
 '우리': 322,
 '것': 1710,
 '모든': 862,
 '지능': 2743,
 '행성': 69,
 '쇼핑': 69,
 '인공': 2628,
 '삶': 147,
 '점프': 156,
 'Do it': 1730,
 '파이썬': 5664,
 '투': 143,
 'Python for everyone': 70,
 '기초': 2458,
 '프로그래밍': 5393,
 '모두': 918,
 '윤성우': 269,
 '혈': 283,
 'C': 1106,
 '김대식': 69,
 'vs': 69,
 '기계': 321,
 '인간': 332,
 '무엇': 190,
 '러닝': 3083,
 '구현': 698,
 '밑바닥': 244,
 '이론': 323,
 '딥': 1931,
 '시작': 2234,
 '수포': 67,
 '이해': 1611,
 '첫걸음': 623,
 '동작': 299,
 '자도': 67,
 '신경망': 244,
 '원리': 1018,
 '현재': 139,
 '미래': 1178,
 '뇌': 232,
 '공학': 104,
 '입문': 4307,
 '머신': 1603,
 '텐서': 474,
 '제대로': 271,
 '회귀분석': 43,
 '군집': 41,
 '합성곱': 41,
 '플로': 471,
 '그림': 1829,
 '개념': 899,
 '엄마': 62,
 '국영수': 56,
 '아이': 317,
 '맨': 150,
 '교과서': 949,
 '처음': 1489,
 '코딩': 5216,
 '컴퓨터': 970,
 '세상': 1172,
 '가지': 1752,
 '아홉': 70,
 '아이디어': 212,
 '언어': 1164,
 'Coding time': 63,
 '지배': 159,
 '시대': 605,
 '코드': 1351,
 '앱': 1023,
 '만들기': 943,
 '첫': 651,
 '위': 4966,
 'CSS': 392,
 '게임': 1749,
 'HTML': 518,
 '홈페이지': 120,
 '자바스크립트

In [ ]:
sr = pd.Series(keyword_dict)

In [ ]:
sr

관                                                125
알고리즘                                            3035
우리                                               322
것                                               1710
모든                                               862
                                                ... 
만점                                                 1
개성                                                 1
마이크로소프트                                            1
계약서                                                1
react modern web development with TypeScript       1
Length: 3057, dtype: int64

In [ ]:
sr.sort_values(ascending=False, inplace=True)

파이썬                                             5664
프로그래밍                                           5393
코딩                                              5216
위                                               4966
입문                                              4307
                                                ... 
워크스페이스                                             1
운영자                                                1
반도체                                                1
원격                                                 1
react modern web development with TypeScript       1
Length: 3057, dtype: int64

TypeError: ignored

In [ ]:
sr_frame = sr.to_frame()
sr_frame

,0
관,125
알고리즘,3035
우리,322
것,1710
모든,862
...,...
만점,1
개성,1
마이크로소프트,1
계약서,1


In [ ]:
sr_frame.sort_values(by=0,ascending=False)

,0
파이썬,5664
프로그래밍,5393
코딩,5216
위,4966
입문,4307
...,...
워크스페이스,1
운영자,1
반도체,1
원격,1


In [ ]:
sr_frame.to_excel("/content/drive/MyDrive/대출 데이터 모음/키워드 처리 폴더/keyword.xlsx")